In [12]:
import numpy as np
import pandas as pd
import re
from nltk.tokenize import word_tokenize

import nltk
from nltk.corpus import stopwords
from nltk import tokenize
from nltk.util import ngrams

from collections import Counter    


from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk.probability import FreqDist
pd.set_option('display.max_colwidth', -1)

<ipython-input-12-69d0281299de>:17: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [13]:
#Stopwords
stop_words=[]
stop_words = (pd.read_excel('C:/Users/Usuario/Desktop/Respositorio archivos .py/NLP/Datasets/stop_words_sin_tilde1.xlsx')) 
stop_words=list(stop_words['stopwords'])

# new_stopwords=['dia','no','si','hábiles','15','días','respuesta','correo','electrónico','autoriza','adjunta','pantallazo','indica','tratamiento','datos','personales'
#               'tiempo', 'gestión','1','usuario','comunica','confronta','exitoso','usuario','solicita','autorización',
#               'ncc','nse','ncorreo','ncedula','ncedula','nnombre','noperación']
# for i in range(0,len(new_stopwords)):
#     stop_words.append(new_stopwords[i])
    
    
eliminar_frases=['usuario se comunica','usuaria se comunica','confronta exitosa','usuario solicita información',
 'usuaria solicita información','se informa que','se le indica correo','se realiza','se solicita','se genera certificación de pagos',
 'se envía la solicitud al correo','por parte del usuariose envía','se genera certificación de pagos','del proyecto','se informa',
 'se brinda','usuaria','solicita información','se informa que','usuario solicita','usuaria solicita','retroalimentacion','se toma solicitud',
 'solicita','usuaria se comunica para manifestar inconformidad','usuario se comunica','se indica','se informa','cédula','celular','solicitud',
 'cajero:','punto:','correo:','errada:','correc:','radicado:','nombre:','cc:','correo:','telefono:','direccion:','radicado:','os:',
 'ps','usuario solicita','pago realizado','buenas tardes','datos personales','respuesta al correo electronico','correo electronico se indica','usuaria solicita',
 'adjunta pantallazo','al correo sac','se lee articulo','se adjunta','pantallazo de correo','confronta no exitoso','usuario autoriza','buenas tardes',
 'cordial saludo','correo electronico','dias habiles','buenos dias','respuesta al correo','autoriza respuesta','el dia de hoy','tiempo de gestion de','quedo atenta','muchas gracias','articulo de la ley','autoriza tiempo',
 'ley','buen dia','pronta respuesta','cedula','numero de referencia','hasta la fecha','el nombre del titular']


#Frases que no aportan valor y no son tenídas en cuenta
# stop_phrases=[' _ _ ',' _ _ ''nombre_del_proyecto','descripción_del_asesor','número_de_caso','realizar_un_pago',
# 'día_de_hoy','usuaria_informa_que','DÍA_DE_HOY','USUARIA_INFORMA_QUE','ASESORA_ELIZABETH_HERNÁNDEZ',
# 'RADICA_ASESORA_ELIZABETH','OPERACIÓN_NOMBRE_DEL','DÍA_DE_AYER','DÍA_DE_HOY']

In [14]:
#Importo
Encuestas = pd.read_excel('C:/Users/Usuario/Desktop/Efecty/Proyecto VOZ/CRM - Dinamycs PQRS/Observaciones de PQRS para frases.xlsx')
# Vista de búsqueda avanzada de casos 29-09-2021 10-33-48.xlsx')

# 
#Encuestas=Encuestas[(Encuestas['Canal de ingreso']=='Web')|(Encuestas['Canal de ingreso']=='Correo electrónico')]
Encuestas.reset_index(inplace=True)
Encuestas.rename(columns={'index':'index_inicial'},inplace=True)
# Encuestas['Descripción']

In [15]:
Encuestas.rename(columns={'DESCRIPCION':'Descripción'},inplace=True)

In [16]:
Encuestas['Descripción']=Encuestas['Descripción'].astype(str)
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: x.lower())
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub('se\s[a-z]{1,10}','',str(x)))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub('á','a',str(x)))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub('é','e',str(x)))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub('í','i',str(x)))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub('ó','o',str(x)))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub('ú','u',str(x)))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub(r"""[\n\t]"""," ",x))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub("""[-*\_¡!@#$:).;–,¿?&'_>/]{1,4}""","",x))
Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub(r"""[0-9]""","",x))

In [17]:
for f in eliminar_frases:
    Encuestas['Descripción']=Encuestas['Descripción'].apply(lambda x: re.sub(f,'',str(x)))

In [18]:
pa_token=re.sub("""[,']""",'',str(list(Encuestas['Descripción'])))
tokenized_word=word_tokenize(pa_token)

In [19]:
# pa_token=re.sub("""[,']""",'',str(list(Encuestas['Descripción'])))
# tokenized_word_0=word_tokenize(pa_token)

#TARDA MUUUUUCHO
# #MISPELL. Ver de agregar las palabras de efecty
# from autocorrect import Speller
# spell = Speller(lang='es')

# tokenized_word=[]
# for r in tokenized_word_0:
#     tokenized_word.append(spell(r)) #No considerar tildes (sacarlas siempre antes)
# # diccionario propio de efecty: efecty, siwewb, TUCUENTA, Portalvirtual,trilla
# # spell('prevalidar')

In [20]:
qty_most_common=30
def gen_frases(qty_words_ngram,max_stopwords,df):
    global qty_most_common
    global Top_words
    Top_words=[]
    Encuestas2=pd.DataFrame()
    ongrams=ngrams(tokenized_word,qty_words_ngram)
    ongrams=list(zip(ongrams))
    ongrams=pd.DataFrame(ongrams)
    ongrams['r']=0
    for i in range(0,qty_words_ngram):
        ongrams['val'+str(i)]=ongrams[0].apply(lambda x: x[i])
        ongrams['w'+str(i)]=0
        index_on_val=ongrams[ongrams['val'+str(i)].isin(stop_words)].index
        ongrams.loc[index_on_val,'w'+str(i)]=1
        ongrams['r']=ongrams['r']+ongrams['w'+str(i)]

    ongrams=ongrams[(ongrams['r']<=max_stopwords)&(ongrams['w'+str(qty_words_ngram-1)]==0)]
    ongrams_=[]
    ongrams.reset_index(inplace=True)

    for i in range(len(ongrams[0])):
        ongrams_.append(re.sub("""[¡!@)#($:.;,¿?&'_]""","",str(ongrams[0][i]))) #Agregar nan

    
    fdist = FreqDist((ongrams_))
    
    df['frases']=''
    for i in range(0,qty_most_common):
        index_frases=df[(df['Descripción'].str.contains(str(fdist.most_common(qty_most_common)[i][0])))&(df['frases']=='')].index
        index_frases_duplicadas=df[(df['Descripción'].str.contains(str(fdist.most_common(qty_most_common)[i][0])))&(df['frases']!='')].index
#Si, veo que esta en el top, pero que no está en las frases df['Descripción'], es porque son 2 palabras que entre medio tenian una frase que 
# se eliminó y queda un espacio grande entre medio que te lo considera dentro del top, pero cuando lo busca no lo encuentra,
# porque no existe tal cual, sino con un espacio de por medio y una frase sin sentido.
        df.loc[index_frases,'frases']=fdist.most_common(qty_most_common)[i][0]
        Top_words.append(fdist.most_common(qty_most_common)[i][0])
        aux2=df.loc[index_frases_duplicadas,:]
        aux2.loc[:,'frases']=fdist.most_common(qty_most_common)[i][0]
        Encuestas2=pd.concat([aux2,Encuestas2])
#     Encuestas2.drop(columns=['level_0'])
    Encuestas2.reset_index(inplace=True)
#     Encuestas2.drop(columns=['level_0'])
    df=pd.concat([df,Encuestas2])
    Encuestas_con_frases=df[df['frases']!='']
#     df['frases']=df['frases'].apply(lambda x: x.upper())
#     Top_words=fdist.most_common(qty_most_common)
    return df


In [21]:
# PRUEBAS!!
# qty_words_ngram=2
# max_stopwords=0
# qty_most_common=20
# df=Encuestas


# global Top_words
# Top_words=[]
# Encuestas2=pd.DataFrame()
# ongrams=ngrams(tokenized_word,qty_words_ngram)
# ongrams=list(zip(ongrams))
# ongrams=pd.DataFrame(ongrams)
# ongrams['r']=0
# for i in range(0,qty_words_ngram):
#     ongrams['val'+str(i)]=ongrams[0].apply(lambda x: x[i])
#     ongrams['w'+str(i)]=0
#     index_on_val=ongrams[ongrams['val'+str(i)].isin(stop_words)].index
#     ongrams.loc[index_on_val,'w'+str(i)]=1
#     ongrams['r']=ongrams['r']+ongrams['w'+str(i)]

# ongrams=ongrams[ongrams['r']<=max_stopwords]
# ongrams_=[]
# ongrams.reset_index(inplace=True)

# for i in range(len(ongrams[0])):
#     ongrams_.append(re.sub("""[¡!@)#($:.;,¿?&'_]""","",str(ongrams[0][i]))) #Agregar nan


# fdist.most_common(qty_most_common)[0][0]
# df[(df['Descripción'].str.contains(str('AUTORIZA')))]
# df['Descripción']

In [22]:
# R_frases1=gen_frases(1,0,20,Encuestas)
# R_frases1=R_frases1[R_frases1['frases']!='']
# top_1_palabra=pd.DataFrame(Top_words)

R_frases2=gen_frases(2,0,Encuestas) 
R_frases2=R_frases2[R_frases2['frases']!='']
top_2_palabras=pd.DataFrame(Top_words)

R_frases3=gen_frases(3,1,Encuestas) 
R_frases3=R_frases3[R_frases3['frases']!='']
top_3_palabras=pd.DataFrame(Top_words)

R_frases4=gen_frases(4,2,Encuestas) 
R_frases4=R_frases4[R_frases4['frases']!='']
top_4_palabras=pd.DataFrame(Top_words)


In [23]:
Encuestas3=pd.concat([Encuestas,R_frases2])
# Encuestas3=pd.concat([Encuestas3,R_frases2])
Encuestas3=pd.concat([Encuestas3,R_frases3])
Encuestas3=pd.concat([Encuestas3,R_frases4])
Encuestas3.sort_values(by='index',inplace=True)
# Encuestas3.drop(columns=['level_0','index'],inplace=True)
Encuestas3.reset_index(inplace=True)

In [24]:
#PROCESO PARA LIMIPAR FRASES SEMEJANTES DENTRO DE 4 PALABRAS. Repetir este proceso para 3 y 2 palabras

eliminar=[]
index_eliminar_2=[]
index_eliminar_3=[]
index_eliminar_4=[]
top_2_palabras_n=top_2_palabras
top_4_palabras_n=top_4_palabras
top_3_palabras_n=top_3_palabras

for j in range(0,qty_most_common): 
    
    #Pensado para 20 palabras mas comunes de 4 palabras. Compara contra las de 4 y las de 3
    qs=re.findall('[\w]{1,15}',top_4_palabras_n[0][j])
    # 4 palabras solo va a ir contra 3
    for i in range(0,len(qs)):
        for p1 in range(0,qty_most_common):
            try: #Mira si hay 3 palabras segúidas repetidas dentro de los 4grams, si es asi lo elimina
                if(((qs[i]+' '+qs[i+1]+' '+qs[i+2]) in top_4_palabras_n[0][p1])&(p1!=j)):
                    eliminar.append(top_4_palabras_n[0][p1])
                    top_4_palabras_n[0][p1]='-4'
                    index_eliminar_4.append(p1)
            except:
                pass

            
            try: #Mira si hay 2 palabras segúidas repetidas dentro de los 3grams, si es asi lo elimina
                if((qs[i]+' '+qs[i+1]) in top_3_palabras_n[0][p1]):
                    eliminar.append(top_3_palabras_n[0][p1])
                    index_eliminar_3.append(p1)
                    top_3_palabras_n[0][p1]='-4'
            except:
                pass

            try: #Mira si hay 2 palabras segúidas repetidas dentro de los 2grams, si es asi lo elimina
                if((qs[i]+' '+qs[i+1]) in top_2_palabras_n[0][p1]):
                    eliminar.append(top_2_palabras_n[0][p1])
                    index_eliminar_2.append(p1)
                    top_2_palabras_n[0][p1]='-4'
            except:
                pass
            
    #Pensado para 20 palabras mas comunes de 3 palabras, compara contra las de 3 y las de 2
    qs3=re.findall('[\w]{1,15}',top_3_palabras_n[0][j])
    # 3 palabras solo va a ir contra 2
    for i in range(0,len(qs3)):
        for p1 in range(0,qty_most_common):
            try: #Mira si hay 3 palabras segúidas repetidas dentro de los fourgrams, si es asi lo elimina
                if(((qs3[i]+' '+qs3[i+1]) in top_3_palabras_n[0][p1])&(p1!=j)):
                    eliminar.append(top_3_palabras_n[0][p1])
                    index_eliminar_3.append(p1)
                    top_3_palabras_n[0][p1]='-3'
            except:
                pass

            try: #Mira si hay 2 palabras segúidas repetidas dentro de los fourgrams, si es asi lo elimina
                if((qs3[i]+' '+qs3[i+1]) in top_2_palabras_n[0][p1]):
                    eliminar.append(top_2_palabras_n[0][p1])
                    index_eliminar_2.append(p1)
                    top_2_palabras_n[0][p1]='-3'
            except:
                pass
            

#ELIMINA LAS FRASES QUE EL PROCESO ELIGIÓ LIMPIAR
for b in range(0,len(eliminar)):
    index_a_eliminar=Encuestas3[Encuestas3['frases']==eliminar[b]].index
    Encuestas3.loc[index_a_eliminar,'frases']='' 


In [25]:
Encuestas3.drop_duplicates(inplace=True)

In [26]:
Encuestas3['frases'].unique()

array(['enviar tirilla', '', 'indica enviar', 'la respuesta la emite',
       'la emite el ce', 'devolucion de os errada',
       'a un proyecto errado', 'proyecto errado pensaba pagar',
       'claro fijo', 'proyecto claro', 'correo sac',
       'informa que por error', 'pago con referencia errada',
       'error generaron', 'normatividad articulo', 'claro movil',
       'relacion de giros', 'novaventa integracion', 'punto de atencion',
       'de tradicion y libertad', 'os errada al proyecto',
       'errada autoriza', 'la devolucion del dinero', 'punto efecty',
       'el dia de ayer', 'la orden de servicio', 'ce envio de tirilla',
       'giros enviados', 'tratamiento de datos', 'me dirijo a ustedes',
       'atencion prestada', 'xd xd', 'p m', 'autoriza tratamiento',
       'confronta exitoso', 'motivo de la llamada', 'gestion autoriza',
       'la expide el ce', 'bajo la os indicando',
       'tiempo de gestion respuesta', 'tirilla articulo', 'ce le envien'],
      dtype=object)

In [27]:
ruta3= "C:/Users/Usuario/Desktop/Respositorio archivos .py/borrador/Frases_pqrs.xlsx"

In [28]:
from pandas import ExcelWriter
from openpyxl import load_workbook

#Ingreso las bases a hojas al archivo
writer = pd.ExcelWriter(ruta3, engine='openpyxl')
Encuestas3[['index_inicial','Descripción','frases']].to_excel(writer, index=True)
writer.save()
writer.close()

In [29]:
#DESDE ACAAA

In [30]:
# https://www.kaggle.com/haibaral/spanish-word2vec/notebook
import gensim
import gensim.downloader as api
from gensim.models import KeyedVectors

In [31]:
model = gensim.models.KeyedVectors.load_word2vec_format('C:/Users/Usuario/Downloads/archive/SBW-vectors-300-min5.txt')

In [32]:
# 'ENVIAR TIRILLA', 'INDICA ENVIAR', 'NUMERO DE CEDULA',
#        'RESPUESTA LA EMITE EL', 'LA EMITE EL CE',
#        'PROYECTO ERRADO PENSABA PAGAR', 'A UN PROYECTO ERRADO',
#        'DEVOLUCION DE OS ERRADA', 'CLARO FIJO', 'PROYECTO CLARO',
#        'CORREO SAC', 'POR ERROR GENERARON', 'POR ERROR GENERARON EL',
#        'ERROR GENERARON EL PAGO', 'INFORMA QUE POR ERROR',
#        'QUE POR ERROR GENERARON', 'CON REFERENCIA ERRADA',
#        'PAGO CON REFERENCIA ERRADA', 'ERROR GENERARON',
#        'ORDEN DE SERVICIO', 'NUMERO DE REFERENCIA',
#        'NORMATIVIDAD ARTICULO', 'TIEMPO DE RESPUESTA',
#        'TIEMPO DE RESPUESTA DE', 'RELACION DE GIROS', 'CLARO MOVIL',
#        'HASTA LA FECHA', 'NOVAVENTA INTEGRACION', 'PUNTO DE ATENCION',
#        'DE TRADICION Y LIBERTAD', 'CON NUMERO DE REFERENCIA',
#        'OS ERRADA AL PROYECTO', 'ERRADA AUTORIZA',
#        'LA DEVOLUCION DEL DINERO', 'CEDULA DE CIUDADANIA',
#        'GIROS ENVIADOS', 'CE ENVIO', 'TRATAMIENTO DE DATOS',
#        'ATENCION PRESTADA', 'XD XD', 'P M', 'AUTORIZA TRATAMIENTO DE',
#        'AUTORIZA TRATAMIENTO', 'CONFRONTA EXITOSO', 'GESTION AUTORIZA',
#        'LA EXPIDE EL CE', 'BAJO LA OS INDICANDO',
#        'TIEMPO DE GESTION RESPUESTA'

In [33]:
model.most_similar('ágil')
# model.words_closer_than('belleza','mujer') #Palabras cercanas a w1 y lejanas a w2
# model.doesnt_match(['deporte','futbol','socer','bota'])
model.n_similarity(['tiempo'],['ellos','no','quieren','porque','hay','mucha','espera','en','trámite'])

# model.most_similar_cosmul(positive='belleza',negative='mujer'). Raro


0.58988434

In [34]:
pip install wikipediaapi

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement wikipediaapi
ERROR: No matching distribution found for wikipediaapi


In [35]:
import nltk
nltk.corpus.gutenberg.words('austen-emma.txt')

LookupError: 
**********************************************************************
  Resource [93mgutenberg[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('gutenberg')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/gutenberg[0m

  Searched in:
    - 'C:\\Users\\Usuario/nltk_data'
    - 'C:\\Users\\Usuario\\anaconda3\\nltk_data'
    - 'C:\\Users\\Usuario\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\Usuario\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Usuario\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
# https://relopezbriega.github.io/blog/2017/09/23/procesamiento-del-lenguaje-natural-con-python/
import wikipedia 
wp = wikipedia( lang='es', version='latest')
wp.download()